In [2]:
import sys
import os

# one level up from the Notebooks directory
notebook_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(notebook_dir)

# Import environment setup
import setup

# Now you can safely import your modules
from data_pipeline.pipeline import EnergyDemandForecasting
import pandas as pd


### For Day-ahead


In [ ]:
ed = EnergyDemandForecasting()

